In [1]:
import websocket
import config
import ccxt
import json
import pandas as pd
from IPython.display import clear_output
from tabulate import tabulate

In [2]:
exchange =  ccxt.binance({
    'apiKey' : config.API_KEY,
    'secret' : config.API_SECRET
})

In [3]:
balanceSpot = exchange.fetch_balance({
    'type' : 'spot'
})
balanceMargin = exchange.fetch_balance({
    'type' : 'margin'
})


In [4]:
symbols = set()
for bal in balanceSpot['info']['balances']:
    if float(bal['free']) > 0:
        symbols.add(bal['asset'])

In [5]:
for val in balanceMargin['info']['userAssets']:
    if float(val['netAsset']) > 0:
        symbols.add(val['asset'])

In [6]:
#symbols.remove('USDT')
print(symbols)

{'ADA', 'MATIC', 'LINK', 'BNB', 'LTC', 'EOS', 'XRP', 'NEO', 'BTC', 'TRX', 'UNI', 'BCH', 'CHZ', 'BAT', 'ETH', 'ETC', 'DOT', 'BTT'}


In [7]:
since = exchange.milliseconds () - 5 * 86400000  # -1 day from now
orders = []
for symbol in symbols:
    SymbolinUSDT = f'{symbol}/USDT'
    orders += exchange.fetch_closed_orders(SymbolinUSDT,since, 10 ,{'type' : 'margin'})

In [8]:
buyOrders = {}
for val in orders:
    if val['info']['side'] == 'BUY':
        symbol = val['info']['symbol'].lower()
        quantity = float(val['info']['executedQty'])
        totalMoneyinUSDT = float(val['info']['cummulativeQuoteQty'])
        if symbol in buyOrders:
            buyOrders[symbol] = {'totalamountinUSDT': buyOrders[symbol]['totalamountinUSDT'] + totalMoneyinUSDT, 'quantity' : buyOrders[symbol]['quantity'] + quantity}
        else:
            buyOrders[symbol] = {'totalamountinUSDT': totalMoneyinUSDT, 'quantity' : quantity}
        #print(val)

In [9]:
print(json.dumps(buyOrders, indent=4))

{
    "adausdt": {
        "totalamountinUSDT": 404.985447,
        "quantity": 230.87
    },
    "maticusdt": {
        "totalamountinUSDT": 450.68318999999997,
        "quantity": 418.0
    },
    "linkusdt": {
        "totalamountinUSDT": 354.929573,
        "quantity": 7.487
    },
    "bnbusdt": {
        "totalamountinUSDT": 254.97185,
        "quantity": 0.415
    },
    "ltcusdt": {
        "totalamountinUSDT": 404.9947174,
        "quantity": 1.12614
    },
    "eosusdt": {
        "totalamountinUSDT": 329.911002,
        "quantity": 30.51
    },
    "xrpusdt": {
        "totalamountinUSDT": 254.995764,
        "quantity": 169.23
    },
    "neousdt": {
        "totalamountinUSDT": 254.886695,
        "quantity": 2.215
    },
    "trxusdt": {
        "totalamountinUSDT": 254.99973,
        "quantity": 2073.0
    },
    "uniusdt": {
        "totalamountinUSDT": 404.96657300000004,
        "quantity": 10.439
    },
    "chzusdt": {
        "totalamountinUSDT": 254.953669,
      

In [10]:
for val in orders:
    if val['info']['side'] == 'BUY':
        symbol = val['info']['symbol'].lower()
        quantity = float(val['info']['executedQty'])
        totalMoneyinUSDT = float(val['info']['cummulativeQuoteQty'])
        print(symbol, quantity, totalMoneyinUSDT)
        # if symbol in buyOrders:
        #     buyOrders[symbol] = {'totalamountinUSDT': buyOrders[symbol]['totalamountinUSDT'] + totalMoneyinUSDT, 'quantity' : buyOrders[symbol]['quantity'] + quantity}
        # else:
        #     buyOrders[symbol] = {'totalamountinUSDT': totalMoneyinUSDT, 'quantity' : quantity}

adausdt 142.48 254.996456
adausdt 88.39 149.988991
maticusdt 208.5 220.74729
maticusdt 209.5 229.9359
linkusdt 5.197 254.959623
linkusdt 2.29 99.96995
bnbusdt 0.415 254.97185
ltcusdt 0.66281 254.9962632
ltcusdt 0.46333 149.9984542
eosusdt 30.51 329.911002
xrpusdt 169.23 254.995764
neousdt 2.215 254.886695
trxusdt 2073.0 254.99973
uniusdt 6.567 254.976909
uniusdt 3.872 149.989664
chzusdt 548.6 254.953669
batusdt 180.95 254.99474
ethusdt 0.0839 349.975426
ethusdt 0.06483 254.9841696


In [11]:
with open('buyOrders.json', 'r') as openfile:
    buyOrders = json.load(openfile)
symbolList = buyOrders.keys()
symbolurl = ""
for x in symbolList:
    symbolurl += f'{x.lower()}@kline_1m/'
symbolurl = symbolurl[:-1]
#print(symbolurl)
socket = f"wss://stream.binance.com:9443/stream?streams={symbolurl}"
#socket = f"wss://stream.binance.com:9443/stream?streams=adausdt@kline_1m"
#print(socket)
#print(buyOrders)

In [16]:
import websocket
data = {}
usdtToINR = 78.25
def on_message(ws, message):
    obj = json.loads(message)
    symbol, cur_price = obj['data']['s'].lower(), float(obj['data']['k']['c'])
    buy_price = float(buyOrders[symbol]['totalamountinUSDT'])
    new_price = cur_price * float(buyOrders[symbol]['quantity'])
    perChange = (new_price - buy_price) * 100 / buy_price

    new_priceINR = new_price * usdtToINR
    new_buypriceINR =  buy_price * usdtToINR
    new_curpriceINR = cur_price * usdtToINR
    data[symbol] = {
            'PercentageChange' : perChange,
            'Current_Wallet' : new_priceINR,
            'Buy_price' : new_buypriceINR, 
            'CurrentPrice' : new_curpriceINR, 
            'Profit/Loss' : new_buypriceINR - new_priceINR
        }
    clear_output(wait=True)
    df = pd.DataFrame(data)
    df = df.T
    df = df.sort_values(by='PercentageChange')
    print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
    print("{0} / {1}".format(int(df['Current_Wallet'].sum(axis=0)), int(df['Buy_price'].sum(axis=0))))

def on_close(ws):
    print("### closed ###")

def on_error(ws, error):
    print(error)

ws = websocket.WebSocketApp(socket ,on_message = on_message, on_close = on_close, on_error=on_error)


In [17]:
ws.run_forever()

AttributeError: 'WebSocket' object has no attribute 'run_forever'

In [ ]:
import json
  
with open("buyOrders.json", "w") as outfile:
    json.dump(buyOrders, outfile)

In [ ]:
df = pd.DataFrame(data)
df = df.T
df = df.sort_values(by='PercentageChange')
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
print("{0} / {1}".format(int(df['Current_Wallet'].sum(axis=0)), int(df['Buy_price'].sum(axis=0))))

In [44]:
import websocket as ws

In [47]:
ws.WebSocketApp

AttributeError: module 'websocket' has no attribute 'WebSocketApp'